In [1]:
import mesa

from mesa import Agent
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import seaborn as sns
import matplotlib.pyplot as plt

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

ModuleNotFoundError: No module named 'mesa'

In [2]:
class HumanAgent(mesa.Agent):
    """This agent is a bitch ass nigga"""

    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position) 

    def step(self):
      self.move()
        
        

In [3]:
class ZombieAgent(mesa.Agent):
    """A retarded stinking zombie"""

    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)


    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)  

        
    def step(self):
      self.move()   
        

In [6]:

class ZombieApocalypse(mesa.Model):
    """A model with some zombies and human agents"""

    def __init__(self, width, height, initial_zombies, initial_humans, seed=69):
        super().__init__(seed=seed)
        self.num_humans = initial_humans
        self.num_zombies = initial_zombies
        self.grid = MultiGrid(width, height, True)

       # Create humans
        for i in range(self.num_humans):
            human = HumanAgent(self)
            x = self.rng.integers(0, self.grid.width)
            y = self.rng.integers(0, self.grid.height)
            self.grid.place_agent(human, (x,y))

        # Create zombies
        for i in range(self.num_zombies):
            zombie = ZombieAgent(self)
            x = self.rng.integers(0, self.grid.width)
            y = self.rng.integers(0, self.grid.height)
            self.grid.place_agent(zombie, (x,y))
            
        # Create DataCollector to track the number of humans and zombies with respective cell content
        self.datacollector = DataCollector(
            model_reporters={"Humans": self.count_humans, "Zombies": self.count_zombies , "coordinates":self.record_cell_contents}
        )  
    def count_humans(self):
         """Count the number of humans in the model."""
         return self.num_humans

    def count_zombies(self):
         """Count the number of zombies in the model."""
         return self.num_zombies
    
    def record_cell_contents(self):
      """Record the contents of each cell in the grid."""
      cell_data = {}
      for cell_contents, (x, y) in self.grid.coord_iter():
        # Store cell coordinates and contents
        cell_data[(x, y)] = [type(agent).__name__ for agent in cell_contents]
      return cell_data
    
    def step(self):
        """Advance the model by one step."""

        self.datacollector.collect(self)
        self.agents.shuffle_do("step")

        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell
            humans = [agent for agent in cell_contents if isinstance(agent, HumanAgent)]
            zombies = [agent for agent in cell_contents if isinstance(agent, ZombieAgent)]

            if humans and zombies:
                for human in humans:
                    # Convert human to zombie
                    self.grid.remove_agent(human)
                    Agent.remove(human)
                    new_zombie = ZombieAgent(self)
                    self.grid.place_agent(new_zombie, (x, y))

                    # Update counters
                    self.num_humans -= 1
                    self.num_zombies += 1
                  

In [7]:
model = ZombieApocalypse(10, 10, 2, 30)
for _ in range(30):  
    model.step()
  

In [ ]:
model_data = model.datacollector.get_model_vars_dataframe() 
print(model_data)


#this won't work as the model dataframe now returns cell contents too

# def plot_population_dynamics(agent_data):
#     """
#     Creates and displays a line plot showing the population dynamics of humans and zombies over time,
#     using the DataFrame index as the time steps.
    
#     Parameters:
#     agent_data (pd.DataFrame): DataFrame containing 'Humans' and 'Zombies' columns,
#                              where the index represents simulation steps
#     """
#     # First, let's make sure we start with a clean plot
#     plt.clf()
    
#     # Set the style for better visualization
#     # 'whitegrid' provides a clean, professional look that makes the data easy to read
#     sns.set_style("whitegrid")
    
#     # Create a new figure with a comfortable size for viewing
#     # 12x6 provides a good aspect ratio for time series data
#     plt.figure(figsize=(12, 6))
    
#     # Prepare the data for plotting by converting the index to a column
#     # This is necessary because seaborn works better with "unpivoted" data
#     plotting_data = agent_data.reset_index()
    
#     # Plot the human population with blue circles
#     sns.lineplot(data=plotting_data, x='index', y='Humans', label='Humans', 
#                 marker='o', markersize=6, linewidth=2, color='blue')
    
#     # Plot the zombie population with red squares
#     sns.lineplot(data=plotting_data, x='index', y='Zombies', label='Zombies',
#                 marker='s', markersize=6, linewidth=2, color='red')
    
#     # Add descriptive labels and title
#     plt.title('Population Dynamics: Humans vs Zombies', fontsize=14, pad=15)
#     plt.xlabel('Simulation Step', fontsize=12)
#     plt.ylabel('Population Count', fontsize=12)
    
#     # Add a legend with clear labels
#     plt.legend(fontsize=10)
    
#     # Ensure all elements fit within the figure bounds
#     plt.tight_layout()
    
#     # Display the plot
#     plt.show()

    
# plot_population_dynamics(agent_data)

    Humans  Zombies                                        coordinates
0       30        2  {(0, 0): [], (0, 1): [], (0, 2): ['ZombieAgent...
1       28        4  {(0, 0): [], (0, 1): [], (0, 2): ['HumanAgent'...
2       26        6  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
3       24        8  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
4       21       11  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
5       20       12  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
6       20       12  {(0, 0): ['ZombieAgent'], (0, 1): [], (0, 2): ...
7       17       15  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
8       15       17  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
9       15       17  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
10      14       18  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
11      12       20  {(0, 0): [], (0, 1): [], (0, 2): [], (0, 3): [...
12      11       21  {(0, 0): [], (0, 1): [], (0, 2): ['ZombieAgent...
13    